# Implementing PageRank
During this practical session, you will implement PageRank for recommendation.

PageRank is decomposed in several steps:
- Create a toy dataset
- Build the item graph
- Implement PageRank
- Run PageRank on the dataset to validate its behavior
- Implement a recommender based on the PageRank
- Adapt the MovieLens validation to use Recall as a validation metric
- Compare it to other approches

## Building a toy dataset
In MovieLens, we know what movies users have seen and how they rated them. For each user, we know when the movie has been seen and what is the rating he gave to it. There are lot of questions here:
- directed on undirected? We have the timestamp of movies so we could have directed edges based on time.
- threshold or not? Even if a user has disliked a movie, he has seen it so it means it matched his interests. Do we want to show movies the user could find interesting, or movies that he would like?
- time window? Should we create links between all the movies a user has seen, or only some of them?

But, for now, we are only interested in the technical function of PageRank. Create a small dataset on which you will be able to iterate fast.

In [109]:
import numpy as np

# Here we simulate the viewing history of 3 users.
# Each line works is as follows:
# - a 0 value indicates that the user has not seen the movie
# - a nonzero value indicates the viewing order of the movie. 1 means first movie seen, 2 means second, etc.

# We want to build the corresponding adjacency matrix.
# For example, for the first user, we want to create the edges:
# 0 -> 1
# 1 -> 3
# 3 -> 4

# Note that this particular format has been chosen to mimic the adjacency matrix available in the wikipedia example

X = [
    [1, 2, 0, 3, 4],
    [1, 0, 2, 0, 3],
    [0, 0, 1, 2, 0],
    [2, 0, 0, 0, 1],
]
X = np.array(X)

## Building the adjacency graph
We want to build the adjacency graph corresponding to this matrix. It will be of shape ni x ni.

In [110]:
import numpy as np

ni = len(X[0])

def build_adjacency_matrix(ratings):
    adjacency_matrix = np.zeros((ni, ni), dtype=int)
    i_sorted = np.argsort(ratings)
    for rating_line in ratings:
        non_zeros_rating_index = np.nonzero(rating_line)[0]
        non_zeros_rating_value = rating_line[non_zeros_rating_index]
        ordered_index_value = np.argsort(non_zeros_rating_value)
        non_zero_argsort_index = non_zeros_rating_index[ordered_index_value]
        adjacency_matrix[non_zero_argsort_index[:-1], non_zero_argsort_index[1:]] +=1
    return adjacency_matrix

adjacency_matrix = build_adjacency_matrix(X)
print(adjacency_matrix)

# Should produce
# [[0 1 1 0 0]
#  [0 0 0 1 0]
#  [0 0 0 1 1]
#  [0 0 0 0 1]
#  [1 0 0 0 0]]

[[0 1 1 0 0]
 [0 0 0 1 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [1 0 0 0 0]]


## Implement PageRank
Reminder: PageRank takes as input an adjacency matrix and outputs a rank for each of the node of the graph. You can implement the stopping criterion as you wish. An interface is proposed but feel free to customize it.

In [180]:
def page_rank(adjacency_matrix, starting_probas=None, g=0.85, n_iter=100, tol=None):
    """Compute the (Personalized) PageRank score corresponding to a given adjacency matrix
    
    Parameters
    ----------
    adjacency_matrix: 2D square numpy array of dimension (n, n)
      The adjacency matrix of the graph on which to compute PageRank score
      
    starting_probas: vector of float, optional
      Vector of probability to "teleport" on each node. By default filled of 1/n, it implements
      PageRank. If given the probability for a user to be on other node, it implements personalized
      PageRank. This vector must be all non-zero.
      
    g: float, optional
      1 - probability of teleporting to a random node
    
    n_iter: int, optional
      Maximum number of iterations
    
    tol: float, optional
      Tolerance. The algorithm stops as soon as the update magnitude of all values is below this threshold.
    """
    n = adjacency_matrix.shape[0]
    if starting_probas is None:
        p = np.ones(n)[:, None] / n
    else:
        p = starting_probas
        p = p/p.sum() #to be sure
    
    last_p = np.full(n, np.inf)
    onces = np.ones(shape=(n,n))
    step = 0
    adjacency_matrix_normalised = adjacency_matrix / np.repeat(adjacency_matrix.sum(axis=1), n).reshape(n,n)
    M = (1-g)/n*onces + g*adjacency_matrix_normalised@np.diag(1/adjacency_matrix_normalised.sum(axis=1))

    while(np.linalg.norm(p - last_p, 2) > tol and step<100 
          if tol is not None
          else step<n_iter):
        p_last = p
        p = M.T@p
        step +=1
    return p
    
    
my_rank = page_rank(adjacency_matrix, n_iter=13)
print(my_rank)

# Should produce
# [[0.25345787]
#  [0.13531093]
#  [0.13531093]
#  [0.20788072]
#  [0.26803955]]

[[0.25345787]
 [0.13531093]
 [0.13531093]
 [0.20788072]
 [0.26803955]]


## Validate PageRank
Now validate you implementation. The best way is to have a reference pageRank matrix with which you can compare yours.

In [181]:
np.allclose(my_rank, np.asarray(
    [[0.25345787],
     [0.13531093],
     [0.13531093],
     [0.20788072],
     [0.26803955]]))

# Should be true

True

## Simple PageRank recommenders
Here are the data available:
- An adjacency matrix: This matrix can contain only 1s and 0s
- A history for a given user: A user history is a set of movies seen by the user represented as their ids. For example {1, 3} corresponds to a user having seen movies 1 and 3. Therefore, we do not want to recommend him neither 1 or 3. User history is a set of products.

We want to do a global recommendation based on PageRank only (no Random Walk).

**Note that we must not recommend any item present in user history for any of the following algorithms.**

### Step 1: Code a general PageRank recommender
If a user has a small history, we want to recommend him the best movie according to a global PageRank.
Tips:
- In this case, the user history is not used to compute the recommendation. However, we need it to avoid recommending a movie that he has already seen

In [182]:
def recommend_pr(adjacency_matrix, user_history, n_products):
    #products = np.arrange(n_products)
    relevants = page_rank(adjacency_matrix).squeeze()
    films_id = np.setdiff1d(np.argsort(relevants), user_history)[:n_products]
    return films_id

recommend_pr(adjacency_matrix, [], 3)

array([0, 1, 2])

### Step 2: Compute a personalized PageRank
Now we will integrate the user history in computation of the PageRank in order to obtain a Personlized PageRank.
For example, among 5 movies, a global PageRank will be run using (0.2, 0.2, 0.2, 0.2, 0.2) as starting probas. For a personalized PageRank, if a user has seen the movie 1, we could compute PageRank using (0.1, 0.6, 0.1, 0.1, 0.1). The weighting of the PageRank is up to you.

In [190]:
def recommend_ppr(adjacency_matrix, user_history, n_products):
#     starting_probas = np.ones(len(adjacency_matrix))
#     starting_probas[user_history] += 2
#     starting_probas = starting_probas / starting_probas.sum()
    
    relevants = page_rank(adjacency_matrix, starting_probas=None).squeeze()
    films_id = np.setdiff1d(np.argsort(relevants), user_history)[:n_products]
    return films_id

recommend_ppr(adjacency_matrix, [1,2], 3)

array([0, 3, 4])

## Introducing a Random Walk
In peculiar usages like spotify where a playlist of music is generated on the fly, we do not want to deviate too much from the previous song. One way to implement this is to do a Random Walk in the graph using PageRank as weighting.

### Step 1: general recommender with Random Walk
The principle is simple. Start points of the random walk are the user history. If user has interacted with items 1 and 3, then the algorithm may start from there. The transition rule is simple:
- Let us say your are on node $i$
- Successors with their corresponding PR value are:
  - $j_1 = 3$
  - $j_2 = 5$
  - $j_3 = 2$
- Then for my Random Walk, my probability to go to:
  - $j_1$ is $0.3$
  - $j_2$ is $0.5$
  - $j_3$ is $0.2$

In [197]:
def recommend_pr_rw(adjacency_matrix, user_history, n_products):
#     starting_probas = np.ones(len(adjacency_matrix))
#     starting_probas[user_history] += 2 
#     starting_probas = starting_probas / starting_probas.sum()
    
    relevants = page_rank(adjacency_matrix, starting_probas=None).squeeze()
    films_id = list()
    vertex = np.random.choice(user_history, 1) #start
    
    for i in range(n_products): #On pourrait faire; on ne s'arrete que quand on a n_products recos, là c'est un maximum.
        linked = adjacency_matrix[vertex].nonzero()[1]
        vertex = np.random.choice(linked, 1, p=relevants[linked]/relevants[linked].sum())
        films_id.append(vertex)
    
    films_id = np.setdiff1d(np.argsort(relevants), user_history)[:n_products]
    return np.array(films_id)

recommend_pr_rw(adjacency_matrix, [1,2], 3)

array([0, 3, 4])

### Step 2: add restart
The problem of the previous approach is that we may end up very far from user preferred products. Code a version of the random walk that has a probability to restart (ie go back to a starting node) with probability $0.1$.

In [179]:
def recommend_pr_rw_with_restart(adjacency_matrix, user_history, n_products):
    starting_probas = np.ones(len(adjacency_matrix))
    starting_probas[user_history] += 0
    starting_probas = starting_probas / starting_probas.sum()
    
    relevants = page_rank(adjacency_matrix, starting_probas=starting_probas).squeeze()
    films_id = list()
    vertex = np.random.choice(user_history, 1) #start
    
    for i in range(n_products): #On pourrait faire; on ne s'arrete que quand on a n_products recos, là c'est un maximum.
        linked = adjacency_matrix[vertex].nonzero()[1]
        vertex = np.random.choice(np.union1d(user_history, linked), 1, p=[0.1/len(user_history) 
                                                                          if v in user_history 
                                                                          else relevants[linked]/relevants[linked].sum()*0.9
                                                                          for v in np.union1d(user_history, linked)])
        films_id.append(vertex)
    
    films_id = np.setdiff1d(np.argsort(relevants), user_history)[:n_products]
    return np.array(films_id)

recommend_pr_rw_with_restart(adjacency_matrix, [1,2], 3)

array([0, 3, 4])

## [Optional] Code the recall metric
Reminder:
    $$\frac{number\_of\_relevant\_items\_predicted}{number\_of\_relevant\_items\_for\_the\_user}$$

In [198]:
def recall(Y_predicted, Y_true):
    return np.sum(Y_predicted == Y_true)/ Y_true

## [Optional] MovieLens evaluation
For the first session, we coded an evaluation framework based on MovieLens. The goal was to infer movie ratings. The setting of the problem has now changed: the goal is to predict 10 products that a user is likely to watch.
Adapt the system to be able to evaluate recall on this problem.